In [1]:
import pandas as pd
from dfply import *
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.decomposition import PCA
print(tf.__version__)

if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

2.1.0
Default GPU Device: /device:GPU:0


In [2]:
df = pd.read_csv("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\behavior-signal-multiple-processed.csv")

In [3]:
#plot = df.plot(legend=False)
#plot.figure.savefig("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\plots\\allplot.png")

In [4]:
#plot = videoDf.plot(legend=False)
#plot.figure.savefig("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\plots\\allVideoPlot.png")

In [3]:
df >>= mutate(drone_present = case_when([df.behavior == 'surround','yes'],
[df.behavior == 'straight','yes'],
[df.behavior == 'noise','no']))
presence_labs = df['drone_present']

In [4]:
values = np.array(presence_labs)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

y.shape

(3200, 2)

In [4]:
x = df.drop(['behavior','signal','multiple','drone_present'],axis=1).values

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
model = keras.Sequential(name='test')
model.add(Dense(64, activation='relu', input_shape=(1280,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(1280,))
model.summary()

Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                81984     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 84,130
Trainable params: 84,130
Non-trainable params: 0
_________________________________________________________________


In [11]:
fit1 = model.fit(x = x_train,
        y = y_train, 
        epochs = 100,
        batch_size = 32,
        validation_split = .2,
        verbose=True)

Train on 2048 samples, validate on 512 samples
Epoch 1/100
2048/2048 [==============================] - 1s 419us/sample - loss: 0.5959 - accuracy: 0.6340 - val_loss: 0.5769 - val_accuracy: 0.6592
Epoch 2/100
2048/2048 [==============================] - 0s 92us/sample - loss: 0.5387 - accuracy: 0.7715 - val_loss: 0.5423 - val_accuracy: 0.8711
Epoch 3/100
2048/2048 [==============================] - 0s 93us/sample - loss: 0.5005 - accuracy: 0.8762 - val_loss: 0.5456 - val_accuracy: 0.8594
Epoch 4/100
2048/2048 [==============================] - 0s 69us/sample - loss: 0.4499 - accuracy: 0.8755 - val_loss: 0.5112 - val_accuracy: 0.8682
Epoch 5/100
2048/2048 [==============================] - 0s 68us/sample - loss: 0.4053 - accuracy: 0.8794 - val_loss: 0.4708 - val_accuracy: 0.8564
Epoch 6/100
2048/2048 [==============================] - 0s 70us/sample - loss: 0.3822 - accuracy: 0.8801 - val_loss: 0.4846 - val_accuracy: 0.8721
Epoch 7/100
2048/2048 [==============================] - 0s 70us

In [12]:
prediction = model.predict(x_test)
prediction_df = pd.DataFrame(prediction, columns=['prob_present','prob_not_present']) 

In [13]:
y_test = pd.DataFrame(y_test,columns=['present','not_present'])

y_pred = y_test >> bind_cols(prediction_df)

y_pred >>= mutate(actual = case_when([y_pred.present == 1, 'present'],
[y_pred.not_present == 1, 'not_present']),
predicted = case_when([y_pred.prob_present > y_pred.prob_not_present, 'present'],
[y_pred.prob_present < y_pred.prob_not_present, 'not_present']))

y_pred >>= mutate(correct = case_when([y_pred.actual == y_pred.predicted, 'yes'],
[y_pred.actual != y_pred.predicted, 'no']))
y_pred


,present,not_present,prob_present,prob_not_present,actual,predicted,correct
0,0.0,1.0,0.000809,0.998866,not_present,not_present,yes
1,1.0,0.0,0.999988,0.000013,present,present,yes
2,0.0,1.0,0.000180,0.999792,not_present,not_present,yes
3,0.0,1.0,0.010557,0.987545,not_present,not_present,yes
4,1.0,0.0,0.999983,0.000019,present,present,yes
...,...,...,...,...,...,...,...
635,0.0,1.0,0.001318,0.998527,not_present,not_present,yes
636,1.0,0.0,0.999997,0.000004,present,present,yes
637,0.0,1.0,0.000009,0.999986,not_present,not_present,yes
638,1.0,0.0,0.999998,0.000003,present,present,yes


In [14]:
df_small = df[df.columns[0:256]]
df_small >>= bind_cols(df.drone_present)

presence_labs = df_small['drone_present']

values = np.array(presence_labs)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

x = df_small.drop(['drone_present'],axis=1).values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = keras.Sequential(name='test')
model.add(Dense(64, activation='relu', input_shape=(256,)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(1280,))
model.summary()

fit1 = model.fit(x = x_train,
        y = y_train, 
        epochs = 100,
        batch_size = 32,
        validation_split = .2,
        verbose=True)

prediction = model.predict(x_test)
prediction_df = pd.DataFrame(prediction, columns=['prob_present','prob_not_present']) 

y_test = pd.DataFrame(y_test,columns=['present','not_present'])

y_pred = y_test >> bind_cols(prediction_df)

y_pred >>= mutate(actual = case_when([y_pred.present == 1, 'present'],
[y_pred.not_present == 1, 'not_present']),
predicted = case_when([y_pred.prob_present > y_pred.prob_not_present, 'present'],
[y_pred.prob_present < y_pred.prob_not_present, 'not_present']))

y_pred >>= mutate(correct = case_when([y_pred.actual == y_pred.predicted, 'yes'],
[y_pred.actual != y_pred.predicted, 'no']))
y_pred



Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 18,594
Trainable params: 18,594
Non-trainable params: 0
_________________________________________________________________
Train on 2048 samples, validate on 512 samples
Epoch 1/100
2048/2048 [==============================] - 1s 290us/sample - lo

,present,not_present,prob_present,prob_not_present,actual,predicted,correct
0,1.0,0.0,0.651111,0.354068,present,present,yes
1,1.0,0.0,0.719579,0.289029,present,present,yes
2,1.0,0.0,0.732523,0.277580,present,present,yes
3,1.0,0.0,0.733460,0.274875,present,present,yes
4,0.0,1.0,0.189913,0.801245,not_present,not_present,yes
...,...,...,...,...,...,...,...
635,1.0,0.0,0.708738,0.296693,present,present,yes
636,0.0,1.0,0.330905,0.664005,not_present,not_present,yes
637,1.0,0.0,0.226199,0.773363,present,not_present,no
638,1.0,0.0,0.654341,0.344885,present,present,yes


In [1]:
x = df.drop(['behavior','signal','multiple','drone_present'],axis=1).values
pca = decomposition.PCA(n_components=1280)
pca.fit(x)
x_transformed = pca.transform(x)
explained_variance = pca.explained_variance_ratio_
sum(explained_variance[0:100])

NameError: name 'df' is not defined

In [6]:
x = df.drop(['behavior','signal','multiple','drone_present'],axis=1).values
pca = decomposition.PCA(n_components=200)
pca.fit(x)
x_transformed = pca.transform(x)

In [7]:
presence_labs = df['drone_present']

pca = decomposition.PCA(n_components=200)
pca.fit(x)
x = pca.transform(x)

values = np.array(presence_labs)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

model = keras.Sequential(name='test')
model.add(Dense(64, activation='relu', input_shape=(200,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(65,))
model.summary()

fit1 = model.fit(x = x_train,
        y = y_train, 
        epochs = 100,
        batch_size = 32,
        validation_split = .2,
        verbose=True)

prediction = model.predict(x_test)
prediction_df = pd.DataFrame(prediction, columns=['prob_present','prob_not_present']) 

y_test = pd.DataFrame(y_test,columns=['present','not_present'])

y_pred = y_test >> bind_cols(prediction_df)

y_pred >>= mutate(actual = case_when([y_pred.present == 1, 'present'],
[y_pred.not_present == 1, 'not_present']),
predicted = case_when([y_pred.prob_present > y_pred.prob_not_present, 'present'],
[y_pred.prob_present < y_pred.prob_not_present, 'not_present']))

y_pred >>= mutate(correct = case_when([y_pred.actual == y_pred.predicted, 'yes'],
[y_pred.actual != y_pred.predicted, 'no']))
y_pred


Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                12864     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 15,010
Trainable params: 15,010
Non-trainable params: 0
_________________________________________________________________
Train on 2048 samples, validate on 512 samples
Epoch 1/100
2048/2048 [==============================] - 1s 401us/sample - loss: 0.7015 - accuracy: 0.5771 - val_loss: 0.6896 - val_accuracy: 0.6016
Epoch 2/100
2048/2048 [==============================] - 0s 119us/sample - loss: 0.6879 - accuracy: 0.6006 - val_loss: 0.6774 - val_accuracy: 0.6064
Epoch 3/100
2048/2048 [====================

,present,not_present,prob_present,prob_not_present,actual,predicted,correct
0,0.0,1.0,0.221175,0.795103,not_present,not_present,yes
1,1.0,0.0,0.985750,0.014067,present,present,yes
2,1.0,0.0,0.979317,0.020947,present,present,yes
3,0.0,1.0,0.186501,0.821260,not_present,not_present,yes
4,0.0,1.0,0.116559,0.879242,not_present,not_present,yes
...,...,...,...,...,...,...,...
635,0.0,1.0,0.183813,0.817890,not_present,not_present,yes
636,1.0,0.0,0.979033,0.020725,present,present,yes
637,0.0,1.0,0.156652,0.835185,not_present,not_present,yes
638,1.0,0.0,0.982812,0.017315,present,present,yes
